In [1]:
from pathlib import Path
from sqlalchemy import create_engine
from langchain.sql_database import SQLDatabase
import sqlite3
from langchain.chains import create_sql_query_chain
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

from dotenv import load_dotenv
import os
import time
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



load_dotenv()

#Langsmith Tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACKING_V2']="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
groq_api_key=os.getenv('GROQ_API_KEY')

llm=ChatGroq(model='llama3-8b-8192', groq_api_key=groq_api_key)

In [17]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessage('''
                      You are a secondary agent who is helping the primary customer care agent.
                      You will be given a user message, if the message contains any user info such as email, phone number, etc.,
                      you should form a question to give it to the tertiary agent who will fetch the user info.
                      If Name, Phone number are given then output something like "Fetch the user info for the user with name and phone number".
                      If customer says to lodge a complaint regarding a specific issue,
                      then output something like Add a record in COMPLAINTS table with for the user
                      DON'T WRITE ANYTHING UNNECESSARY.
                      '''),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [18]:
chain=prompt|llm

In [19]:
response=chain.invoke(
    [HumanMessage("I want to lodge a complaint regarding the issue I faced with the product")]
)

response.content

'Add a record in COMPLAINTS table with the issue faced with the product.'